In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.5.0"
#r "nuget:Microsoft.ML.FastTree,1.5.0"
#r "nuget:Microsoft.ML.AutoML,0.17.2"
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML.AutoML version 0.17.2

Installed package Microsoft.ML version 1.5.0

Installed package Microsoft.ML.FastTree version 1.5.0

Installed package XPlot.Plotly version 2.0.0

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.FastTree;
using Microsoft.ML.AutoML;
using XPlot.Plotly;

In [3]:
public class TransactionObservation 
    {
        [LoadColumn(1)]
        public float V1{ get; set; }

        [LoadColumn(2)]
        public float V2{ get; set; }

        [LoadColumn(3)]
        public float V3{ get; set; }

        [LoadColumn(4)]
        public float V4{ get; set; }

        [LoadColumn(5)]
        public float V5{ get; set; }

        [LoadColumn(6)]
        public float V6{ get; set; }

        [LoadColumn(7)]
        public float V7{ get; set; }

        [LoadColumn(8)]
        public float V8{ get; set; }

        [LoadColumn(9)]
        public float V9{ get; set; }

        [LoadColumn(10)]
        public float V10{ get; set; }

        [LoadColumn(11)]
        public float V11{ get; set; }

        [LoadColumn(12)]
        public float V12{ get; set; }

        [LoadColumn(13)]
        public float V13{ get; set; }

        [LoadColumn(14)]
        public float V14{ get; set; }

        [LoadColumn(15)]
        public float V15{ get; set; }

        [LoadColumn(16)]
        public float V16{ get; set; }

        [LoadColumn(17)]
        public float V17{ get; set; }

        [LoadColumn(18)]
        public float V18{ get; set; }

        [LoadColumn(19)]
        public float V19{ get; set; }

        [LoadColumn(20)]
        public float V20{ get; set; }

        [LoadColumn(21)]
        public float V21{ get; set; }

        [LoadColumn(22)]
        public float V22{ get; set; }

        [LoadColumn(23)]
        public float V23{ get; set; }

        [LoadColumn(24)]
        public float V24{ get; set; }

        [LoadColumn(25)]
        public float V25{ get; set; }

        [LoadColumn(26)]
        public float V26{ get; set; }

        [LoadColumn(27)]
        public float V27{ get; set; }

        [LoadColumn(28)]
        public float V28{ get; set; }

        [LoadColumn(29)]
        public float Amount{ get; set; }

        [LoadColumn(30)]
        public bool Label{ get; set; }
    }

In [4]:
public class TransactionFraudPrediction 
    {
        [ColumnName("PredictedLabel")]
        public bool Prediction { get; set; }
        public float Score { get; set; }
        public float Probability { get; set; }
    }

In [5]:
private void CreateRow(string message, int width)
{
    display("|" + message.PadRight(width - 2) + "|");
}

In [6]:
private void PrintBinaryClassificationMetricsHeader()
{
    CreateRow($"{"",-4} {"Trainer",-35} {"Accuracy",9} {"AUC",8} {"AUPRC",8} {"F1-score",9} {"Duration",9}", 114);
}

In [7]:
private void PrintIterationMetrics(int iteration, string trainerName, BinaryClassificationMetrics metrics, double? runtimeInSeconds)
{
    CreateRow($"{iteration,-4} {trainerName,-35} {metrics?.Accuracy ?? double.NaN,9:F4} {metrics?.AreaUnderRocCurve ?? double.NaN,8:F4} {metrics?.AreaUnderPrecisionRecallCurve ?? double.NaN,8:F4} {metrics?.F1Score ?? double.NaN,9:F4} {runtimeInSeconds.Value,9:F1}", 114);
}

In [8]:
public static void PrintBinaryClassificationMetrics(string name, BinaryClassificationMetrics metrics)
{
            display($"************************************************************");
            display($"*       Metrics for {name} binary classification model      ");
            display($"*-----------------------------------------------------------");
            display($"*       Accuracy: {metrics.Accuracy:P2}");
            display($"*       Area Under Curve:      {metrics.AreaUnderRocCurve:P2}");
            display($"*       Area under Precision recall Curve:  {metrics.AreaUnderPrecisionRecallCurve:P2}");
            display($"*       F1Score:  {metrics.F1Score:P2}");
            display($"*       PositivePrecision:  {metrics.PositivePrecision:#.##}");
            display($"*       PositiveRecall:  {metrics.PositiveRecall:#.##}");
            display($"*       NegativePrecision:  {metrics.NegativePrecision:#.##}");
            display($"*       NegativeRecall:  {metrics.NegativeRecall:P2}");
            display($"************************************************************");
}

In [9]:
string dataFile = "Data/creditcard.csv";

In [10]:
MLContext mlContext = new MLContext(seed: 1);

In [11]:
//Load the original single dataset
IDataView originalFullData = mlContext.Data.LoadFromTextFile<TransactionObservation>
(dataFile, separatorChar: ',', hasHeader: true);

In [12]:
using static Microsoft.ML.DataOperationsCatalog;
// Split the data 80:20 into train and test sets, train and evaluate.
TrainTestData trainTestData = mlContext.Data.TrainTestSplit(originalFullData, testFraction: 0.2, seed: 1);
IDataView trainData = trainTestData.TrainSet;
IDataView testData = trainTestData.TestSet;

In [13]:
var preViewTransformedData = trainData.Preview(maxRows: 4);

foreach (var row in preViewTransformedData.RowView)
{
    var ColumnCollection = row.Values;
    string lineToPrint = "Row--> ";
    foreach (KeyValuePair<string, object> column in ColumnCollection)
    {
        lineToPrint += $"| {column.Key}:{column.Value}";
    }
    display(lineToPrint + "\n");
}

Row--> | V1:-1.3598071| V2:-0.072781175| V3:2.5363467| V4:1.3781552| V5:-0.33832076| V6:0.46238777| V7:0.23959856| V8:0.0986979| V9:0.36378697| V10:0.09079417| V11:-0.55159956| V12:-0.61780083| V13:-0.9913899| V14:-0.31116936| V15:1.468177| V16:-0.4704005| V17:0.20797125| V18:0.02579058| V19:0.40399295| V20:0.2514121| V21:-0.018306779| V22:0.27783757| V23:-0.11047391| V24:0.066928074| V25:0.12853935| V26:-0.18911484| V27:0.13355838| V28:-0.021053053| Amount:149.62| Label:False| SamplingKeyColumn:0.81377757


Row--> | V1:1.1918571| V2:0.2661507| V3:0.16648011| V4:0.4481541| V5:0.06001765| V6:-0.08236081| V7:-0.07880298| V8:0.08510166| V9:-0.25542513| V10:-0.16697441| V11:1.6127267| V12:1.0652353| V13:0.489095| V14:-0.14377229| V15:0.63555807| V16:0.46391705| V17:-0.11480466| V18:-0.18336128| V19:-0.14578304| V20:-0.06908313| V21:-0.22577524| V22:-0.63867193| V23:0.10128802| V24:-0.33984646| V25:0.1671704| V26:0.12589453| V27:-0.008983099| V28:0.014724169| Amount:2.69| Label:False| SamplingKeyColumn:0.8037591


Row--> | V1:-0.9662717| V2:-0.18522601| V3:1.7929933| V4:-0.86329126| V5:-0.010308879| V6:1.2472031| V7:0.23760894| V8:0.37743586| V9:-1.387024| V10:-0.05495192| V11:-0.22648726| V12:0.17822823| V13:0.5077569| V14:-0.28792375| V15:-0.6314181| V16:-1.0596472| V17:-0.68409276| V18:1.965775| V19:-1.232622| V20:-0.20803778| V21:-0.108300455| V22:0.005273597| V23:-0.19032052| V24:-1.1755754| V25:0.64737606| V26:-0.22192885| V27:0.06272285| V28:0.06145763| Amount:123.5| Label:False| SamplingKeyColumn:0.45860124


Row--> | V1:-1.158233| V2:0.87773675| V3:1.5487179| V4:0.40303394| V5:-0.40719336| V6:0.095921464| V7:0.59294075| V8:-0.27053267| V9:0.8177393| V10:0.7530744| V11:-0.8228429| V12:0.53819555| V13:1.3458515| V14:-1.1196698| V15:0.17512113| V16:-0.4514492| V17:-0.23703323| V18:-0.038194787| V19:0.80348694| V20:0.40854236| V21:-0.009430697| V22:0.7982785| V23:-0.13745809| V24:0.14126699| V25:-0.20600958| V26:0.5022922| V27:0.21942224| V28:0.21515314| Amount:69.99| Label:False| SamplingKeyColumn:0.84411216



warning CS1701: Assuming assembly reference 'System.Collections.Immutable, Version=1.2.3.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'Microsoft.ML.Data' matches identity 'System.Collections.Immutable, Version=5.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Collections.Immutable', you may need to supply runtime policy



In [14]:
var preViewTransformedData = testData.Preview(maxRows: 4);

foreach (var row in preViewTransformedData.RowView)
{
    var ColumnCollection = row.Values;
    string lineToPrint = "Row--> ";
    foreach (KeyValuePair<string, object> column in ColumnCollection)
    {
        lineToPrint += $", {column.Key}={column.Value}F";
    }
    display(lineToPrint + "\n");
}

Row--> , V1=-1.3583541F, V2=-1.3401631F, V3=1.7732093F, V4=0.3797796F, V5=-0.50319815F, V6=1.8004994F, V7=0.79146093F, V8=0.24767579F, V9=-1.5146543F, V10=0.20764287F, V11=0.62450147F, V12=0.066083685F, V13=0.7172927F, V14=-0.16594592F, V15=2.345865F, V16=-2.8900833F, V17=1.1099694F, V18=-0.12135931F, V19=-2.261857F, V20=0.5249797F, V21=0.24799815F, V22=0.7716794F, V23=0.90941226F, V24=-0.6892809F, V25=-0.32764184F, V26=-0.13909657F, V27=-0.055352796F, V28=-0.059751842F, Amount=378.66F, Label=FalseF, SamplingKeyColumn=0.14511466F


Row--> , V1=1.2499987F, V2=-1.2216368F, V3=0.38393015F, V4=-1.2348987F, V5=-1.4854195F, V6=-0.75323015F, V7=-0.68940496F, V8=-0.22748722F, V9=-2.0940106F, V10=1.3237293F, V11=0.22766623F, V12=-0.242682F, V13=1.2054168F, V14=-0.31763053F, V15=0.725675F, V16=-0.8156122F, V17=0.8739365F, V18=-0.8477886F, V19=-0.6831926F, V20=-0.10275594F, V21=-0.23180924F, V22=-0.48328534F, V23=0.08466769F, V24=0.39283088F, V25=0.16113456F, V26=-0.35499004F, V27=0.02641555F, V28=0.04242209F, Amount=121.5F, Label=FalseF, SamplingKeyColumn=0.043524146F


Row--> , V1=1.0693736F, V2=0.28772214F, V3=0.82861274F, V4=2.7125204F, V5=-0.17839801F, V6=0.33754373F, V7=-0.09671686F, V8=0.115981735F, V9=-0.22108257F, V10=0.46023044F, V11=-0.7736569F, V12=0.32338724F, V13=-0.011075887F, V14=-0.17848517F, V15=-0.65556425F, V16=-0.19992517F, V17=0.124005415F, V18=-0.9804962F, V19=-0.98291606F, V20=-0.15319723F, V21=-0.03687553F, V22=0.074412405F, V23=-0.07140743F, V24=0.10474375F, V25=0.54826474F, V26=0.104094155F, V27=0.021491058F, V28=0.021293312F, Amount=27.5F, Label=FalseF, SamplingKeyColumn=0.15626693F


Row--> , V1=-0.41428882F, V2=0.90543735F, V3=1.727453F, V4=1.4734713F, V5=0.007442741F, V6=-0.20033067F, V7=0.7402283F, V8=-0.0292474F, V9=-0.593392F, V10=-0.34618822F, V11=-0.012142189F, V12=0.78679633F, V13=0.6359539F, V14=-0.08632447F, V15=0.076803684F, V16=-1.4059193F, V17=0.77559173F, V18=-0.9428889F, V19=0.54396945F, V20=0.09730759F, V21=0.077237435F, V22=0.45733058F, V23=-0.038499724F, V24=0.6425219F, V25=-0.18389134F, V26=-0.27746403F, V27=0.18268749F, V28=0.15266465F, Amount=33F, Label=FalseF, SamplingKeyColumn=0.039589286F



warning CS1701: Assuming assembly reference 'System.Collections.Immutable, Version=1.2.3.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'Microsoft.ML.Data' matches identity 'System.Collections.Immutable, Version=5.0.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Collections.Immutable', you may need to supply runtime policy



In [15]:
//Run AutoML binary classification experiment
ExperimentResult<BinaryClassificationMetrics> experimentResult = mlContext.Auto()
                .CreateBinaryClassificationExperiment(60)
                .Execute(trainData);

In [16]:
// Get top few runs ranked by accuracy
var topRuns = experimentResult.RunDetails
                .Where(r => r.ValidationMetrics != null && !double.IsNaN(r.ValidationMetrics.Accuracy))
                .OrderByDescending(r => r.ValidationMetrics.Accuracy).ToList();

display("Top models ranked by accuracy --");
PrintBinaryClassificationMetricsHeader();
for (var i = 0; i < topRuns.Count(); i++)
{
    var run = topRuns.ElementAt(i);
    PrintIterationMetrics(i + 1, run.TrainerName, run.ValidationMetrics, run.RuntimeInSeconds);
}

Top models ranked by accuracy --

|     Trainer                              Accuracy      AUC    AUPRC  F1-score  Duration                        |

|1    LightGbmBinary                         0.9995   0.9207   0.7932    0.8182       8.2                        |

|2    FastForestBinary                       0.9995   0.9600   0.8039    0.8065       9.2                        |

|3    AveragedPerceptronBinary               0.9992   0.9648   0.6408    0.7187       6.8                        |

...

|5    LbfgsLogisticRegressionBinary          0.9990   0.9902   0.5980    0.6333       7.3                        |

|6    SgdCalibratedBinary                    0.9990   0.9802   0.5835    0.6333       6.9                        |

|7    SdcaLogisticRegressionBinary           0.9990   0.9822   0.5919    0.6102       5.7                        |

|8    SymbolicSgdLogisticRegressionBinary    0.9986   0.8838   0.5591    0.3600       5.9                        |

|9    LinearSvmBinary                        0.9986   0.9368   0.5771    0.3265       5.8                        |

In [17]:
//Evaluate the model and print metrics
display("=============== Evaluating model's accuracy with test data ===============");
RunDetail<BinaryClassificationMetrics> bestRun = experimentResult.BestRun;
ITransformer trainedModel = bestRun.Model;
var predictions = trainedModel.Transform(testData);
var metrics = mlContext.BinaryClassification.EvaluateNonCalibrated(data:predictions, scoreColumnName: "Score");
PrintBinaryClassificationMetrics(bestRun.TrainerName, metrics);

=============== Evaluating model's accuracy with test data ===============

************************************************************

*       Metrics for LightGbmBinary binary classification model      

*-----------------------------------------------------------

*       Accuracy: 99.95%

...

*       PositivePrecision:  .9

*       PositiveRecall:  .78

*       NegativePrecision:  1

*       NegativeRecall:  99.98%

************************************************************

In [18]:
bestRun.Estimator 

LastEstimator
Microsoft.ML.Trainers.LightGbm.LightGbmBinaryTrainer


In [20]:
TransactionObservation singleData = new TransactionObservation(){
    V1=-1.3583541F, V2=-1.3401631F, V3=1.7732093F, V4=0.3797796F, V5=-0.50319815F, V6=1.8004994F, V7=0.79146093F, 
    V8=0.24767579F, V9=-1.5146543F, V10=0.20764287F, V11=0.62450147F, V12=0.066083685F, V13=0.7172927F, 
    V14=-0.16594592F, V15=2.345865F, V16=-2.8900833F, V17=1.1099694F, V18=-0.12135931F, V19=-2.261857F, 
    V20=0.5249797F, V21=0.24799815F, V22=0.7716794F, V23=0.90941226F, V24=-0.6892809F, V25=-0.32764184F, 
    V26=-0.13909657F, V27=-0.055352796F, V28=-0.059751842F, Amount=378.66F, Label=false 
    //SamplingKeyColumn=0.14511466F
};

In [21]:
//Get all the feature column names (All except the Label and the IdPreservationColumn)
string[] featureColumnNames = trainData.Schema.AsQueryable()
.Select(column => column.Name)                               // Get alll the column names
.Where(name => name != nameof(TransactionObservation.Label)) // Do not include the Label column
.Where(name => name != "IdPreservationColumn")
.Where(name => name != "SamplingKeyColumn")
.ToArray();

In [22]:
// Create the data process pipeline
IEstimator<ITransformer> dataProcessPipeline = mlContext.Transforms.Concatenate("Features", featureColumnNames)
.Append(mlContext.Transforms.NormalizeMeanVariance
                        (inputColumnName: "Features",
                        outputColumnName: "FeaturesNormalizedByMeanVar"));


(2,48): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.9.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



In [23]:
var trainer =mlContext.BinaryClassification.Trainers.LightGbm(
        labelColumnName: nameof(TransactionObservation.Label),
        featureColumnName: "FeaturesNormalizedByMeanVar", numberOfLeaves: 20, minimumExampleCountPerLeaf: 10,
        learningRate: 0.2,numberOfIterations:1000);

In [24]:
var trainingPipeline = dataProcessPipeline.Append(trainer);


(1,24): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.9.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



In [25]:
var model = trainingPipeline.Fit(trainData);

In [26]:
var fccPipeline = model.Append(mlContext.Transforms
                .CalculateFeatureContribution(model.LastTransformer)
                .Fit(dataProcessPipeline.Fit(trainData).Transform(trainData)));

In [27]:
// Evaluate the model and show accuracy stats
var predictions = fccPipeline.Transform(testData);

In [28]:
var metrics = mlContext.BinaryClassification.Evaluate(data: predictions, 
    labelColumnName: nameof(TransactionObservation.Label), 
    scoreColumnName: "Score");

In [29]:
PrintBinaryClassificationMetrics("LightGbm", metrics);

************************************************************

*       Metrics for LightGbm binary classification model      

*-----------------------------------------------------------

*       Accuracy: 99.95%

*       Area Under Curve:      94.99%

...

*       PositivePrecision:  .91

*       PositiveRecall:  .77

*       NegativePrecision:  1

*       NegativeRecall:  99.99%

************************************************************

In [30]:
// Create prediction engine related to the loaded trained model
var predEngine= mlContext.Model.CreatePredictionEngine<TransactionObservation, TransactionFraudPrediction>(fccPipeline);

In [31]:
// Score
var predictedResult = predEngine.Predict(singleData);

In [32]:
display(Convert.ToBoolean(predictedResult.Prediction) ? "Fraud" : "Non Fraud")

Non Fraud